# Face Recognition From a Dataset

### Part 1 _ Create dataset

If you saw my last notebook about face recognition, I compared an image with other images in a folder and decided 'who is in photo'. As I mentioned at the end of that notebook, encoding each faces in a folder is not a proper way for face recognition. There are two main reasons. One of them,face_recognition library uses dlib and it takes time to encode faces. And also for a face recognition project, we should have much more images for a person. 

In this notebook, We will have more images for each person and we will save the encodings of faces in a file to use when we make a comparation. By this way, we will get rid of the most time consuming part of face recognition. We will create a dataset and save it. When we need to make a face recognition, we don't need to apply encoding part again. 

As we compare a face with lots of images, algorithm will return lots of 'true' for a face. For this case, we will count 'how many times the name is predicted for the face' and will take name with maximum votes as the last precition.

As always, This notebook is a collection of my internet research. All the images and codes are available in the reference websites. I just prepare it for myself as a reminder and for someone who is eager to learn these concepts.


In [7]:
# first import libraries
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import numpy as np
print('PACKAGES ARE IMPORTED....')

PACKAGES ARE IMPORTED....


We should write the path of our dataset folder and then we need to take each image. For this purpose, i used paths(from imutils) and print results

In [5]:
images_path = list(paths.list_images('photos'))
print(image_path)



['photos\\alan_grant\\00000000.jpg', 'photos\\alan_grant\\00000002.jpg', 'photos\\alan_grant\\00000004.png', 'photos\\alan_grant\\00000005.jpg', 'photos\\alan_grant\\00000006.jpg', 'photos\\alan_grant\\00000009.png', 'photos\\alan_grant\\00000015.JPG', 'photos\\alan_grant\\00000017.jpg', 'photos\\alan_grant\\00000021.jpg', 'photos\\alan_grant\\00000022.jpg', 'photos\\alan_grant\\00000023.jpg', 'photos\\alan_grant\\00000024.jpg', 'photos\\alan_grant\\00000025.png', 'photos\\alan_grant\\00000027.png', 'photos\\alan_grant\\00000028.png', 'photos\\alan_grant\\00000031.jpg', 'photos\\alan_grant\\00000041.JPG', 'photos\\alan_grant\\00000046.jpg', 'photos\\alan_grant\\00000063.jpg', 'photos\\alan_grant\\00000065.png', 'photos\\alan_grant\\00000082.jpg', 'photos\\alan_grant\\00000089.jpeg', 'photos\\claire_dearing\\00000000.png', 'photos\\claire_dearing\\00000001.jpg', 'photos\\claire_dearing\\00000002.jpg', 'photos\\claire_dearing\\00000003.jpg', 'photos\\claire_dearing\\00000004.png', 'photo

In [4]:
# create two list for encodings and names,
# we will collect them in these lists 
face_Encodings = []
face_Names = []

Now we should take encodings of faces in each image and save encodings and names of faces in the lists that we created. For this purpose, we will write a for loop to apply this encodings and saving process to each image in the dataset.(this takes time on cpu, don't worry)

In [6]:
for (i, imagePath) in enumerate(images_path):
    # to extract the person name from the image path, use split method.
    
    name = imagePath.split(os.path.sep)[-2]
    #print(name)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # detect the coordinates of the bounding boxes for faces in image
    boxes = face_recognition.face_locations(rgb)
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    for encoding in encodings:
        # add each encoding and name to our set of lists
        face_Encodings.append(encoding)
        face_Names.append(name)

In [12]:
print(face_Encodings)
print(face_Names)

[array([-6.91878051e-02,  6.56917617e-02, -9.16517898e-03, -4.57990170e-02,
       -8.15012529e-02, -7.00244531e-02,  5.65325469e-02, -6.75042197e-02,
        1.45259306e-01,  7.80347139e-02,  2.29809627e-01, -6.91129118e-02,
       -3.10542136e-01, -8.99551883e-02, -2.07676291e-02,  6.23493046e-02,
       -1.98097363e-01, -3.42449248e-02, -1.73754841e-01, -4.98856120e-02,
        1.24503046e-01,  5.48074208e-02,  5.81974201e-02,  5.10589294e-02,
       -1.84507161e-01, -2.84769475e-01, -3.56605202e-02, -1.28170639e-01,
        6.70389533e-02, -5.16755879e-02,  1.06088474e-01,  1.00849904e-01,
       -1.76399827e-01, -9.97670814e-02,  4.63782474e-02,  1.24166142e-02,
       -9.34348926e-02, -2.90064141e-02,  2.60582983e-01, -5.35611287e-02,
       -1.89174503e-01,  6.82004588e-03,  8.90145302e-02,  2.26055428e-01,
        1.68259963e-01,  7.59437382e-02,  3.12701650e-02, -1.66534223e-02,
        1.28275558e-01, -3.25665623e-01,  2.67130695e-02,  2.29451329e-01,
        1.23633713e-01, 

In [9]:
print(np.shape(face_Encodings))
print(np.shape(face_Names))

(179, 128)
(179,)


We have 179 names and 128 dims encodings for each of them. Let's write them on a file for using later.

In [10]:
data = {"encodings": face_Encodings, "names": face_Names}
f = open('embedings_for_all_images', "wb")
f.write(pickle.dumps(data))
f.close()

If you follow these steps, you should have a file in the same directory with the name of 'embedings_for_all_images'. This file has all embedings and names for each of them. Now we are ready to compare an image with our dataset.

### PART 2 _ Name Prediction

First load our embedings file to use face fecognition

In [13]:
all_faces = pickle.loads(open('embedings_for_all_images', "rb").read())

Then load our test image, convert it to rgb channel for face_recognition and collect faces in the image. An image may have more than one faces, because of that reason i used 'faces', not face. After finding faces, compute the encodings. If you want detailed info about these process you can check my earlier notebooks or reference links.

In [14]:
# load the input image and convert it from BGR to RGB
image = cv2.imread('example_01.png')
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

boxes = face_recognition.face_locations(rgb)
print(boxes)
encodings = face_recognition.face_encodings(rgb, boxes)
print(encodings)


[(56, 474, 163, 366), (80, 175, 187, 67)]
[array([-1.66109413e-01,  2.64765546e-02,  7.74895325e-02, -2.43143979e-02,
       -9.39485580e-02, -6.10842369e-02, -8.65109414e-02, -7.09449872e-02,
        1.88277200e-01, -6.16820157e-02,  1.76950350e-01,  4.33558971e-02,
       -1.33706361e-01, -8.66893753e-02, -5.89717850e-02,  5.15753739e-02,
       -1.58904061e-01, -1.23800442e-01, -6.00119121e-02, -1.19104490e-01,
        2.59850901e-02,  7.38557847e-03, -8.89135152e-03, -2.73951283e-03,
       -2.65886366e-01, -3.26826930e-01, -1.08362272e-01, -1.36471272e-01,
        8.00130665e-02, -1.06286362e-01,  9.36385915e-02,  2.53955051e-02,
       -2.14302897e-01, -4.63441275e-02,  3.97716947e-02,  1.26050323e-01,
       -3.18275131e-02,  4.57475781e-02,  1.88235298e-01,  1.51312165e-03,
       -1.55779347e-01,  3.36500816e-04, -9.16268677e-04,  2.41986975e-01,
        1.96666643e-01,  7.81256258e-02,  4.50054035e-02, -1.12233967e-01,
        1.70592785e-01, -1.75744727e-01,  1.01027548e-01,

In [18]:
# initialize the list of names for each face detected
names = []

In [19]:
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(all_faces["encodings"],encoding)
    # matches list will have True or False for each(179) comparation 
    print(matches)
    name = "Unknown"

    # check if we have True(it means the algorithm found it is a person from dataset)
    if True in matches:
        # take the indexes of 'true' prediction for the name of encodings
        
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        print(matchedIdxs)
        #then create a dictionary to count how many true for each names
        #Because we will decide who is in image according to largest number of votes
        counts = {}

        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            name = all_faces["names"][i]
            print(name)
            counts[name] = counts.get(name, 0) + 1
        print(counts)
        # determine the recognized face with the largest number of votes
        name = max(counts, key=counts.get)
    names.append(name)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F



For clarification, I used 'print' comment to show results.

An important note : the  'max(counts, key=counts.get)' command will return the first value in the dictionary if there is a tie. Be careful about that problem.

Now we have coordinates of faces and prediction for each faces. Let's show them on the image with a classical opencv methods.

In [20]:
for ((top, right, bottom, left), name) in zip(boxes, names):
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    cv2.putText(image, name, (left, top), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)


# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

-1

Reference Links:

* https://pyimagesearch.com/2018/06/18/face-recognition-with-opencv-python-and-deep-learning/



